### Reinitialize the file

In [1]:
# warning !! remove all except this script
## remove -i argument to run
!rm `(ls | grep -v '.ipynb')`

In [2]:
pdbid = '1kf1'
drug  = 'POH'
jobname = f'{pdbid}_{drug}' if not drug is None else pdbid

In [3]:
import os
ff_folder  = os.path.realpath("../../../smsl/opt/force_field")
mdp_folder = os.path.realpath("../../../smsl/opt/mdp")

### Copy DNA from the file which process CHARMM

In [4]:
if drug is None:
    !cp ../../a.charmm.dna_setup/{jobname}.pgro.pdb ./
else: 
    !cp ../../b.amber.drug_ff/5.Output/{pdbid}.pgro.pdb ./
    !cp ../../b.amber.drug_ff/5.Output/*{drug}* ./

In [5]:
!export GMXLIB={ff_folder};\
 echo "2" | gmx pdb2gmx -f {pdbid}.pgro.pdb -o {pdbid}.gro.pdb -chainsep ter -water tip3p

                     :-) GROMACS - gmx pdb2gmx, 2023.2 (-:

Executable:   /home/user/opt/gromacs-2023/bin/gmx
Data prefix:  /home/user/opt/gromacs-2023
Working dir:  /home/user/workfolder/g4dna_protocol/propeller_poh/c.gromacs.run_md/1.create_system
Command line:
  gmx pdb2gmx -f 1kf1.pgro.pdb -o 1kf1.gro.pdb -chainsep ter -water tip3p

Select the Force Field:

From '/home/user/workfolder/g4dna_protocol/smsl/opt/force_field':

 1: AMBER14SB_parmbsc1

 2: OL21 nucleic (Zgarbova et al., J. Chem. Theory Comput. 17, 6292-6301, 2021

From '/home/user/opt/gromacs-2023/share/gromacs/top':

 3: AMBER03 protein, nucleic AMBER94 (Duan et al., J. Comp. Chem. 24, 1999-2012, 2003)

 4: AMBER94 force field (Cornell et al., JACS 117, 5179-5197, 1995)

 5: AMBER96 protein, nucleic AMBER94 (Kollman et al., Acc. Chem. Res. 29, 461-469, 1996)

 6: AMBER99 protein, nucleic AMBER94 (Wang et al., J. Comp. Chem. 21, 1049-1074, 2000)

 7: AMBER99SB protein, nucleic AMBER94 (Hornak et al., Proteins 65, 712-725

### Modify `topol.top`
1. in line 29(Include forcefield parameters下兩行), Adding
```
; Include ligand parameters
#include "$drug_GMX.top"
```
2. in line 36(Include water topology之上), Adding
```
; Include ligand topology
#ifdef POSRES
#include "posre_$drug.itp"
#endif
```
3. in line 61(最後一行), Adding
```
$drug                 1
(remember to reserved space here)
```

In [6]:
# Step 1: Add the ligand parameters include line at line 29
!sed -i "28a ; Include ligand parameters\n#include \"\{drug}_GMX.top\"\n" topol.top

# # Step 2: Add the ligand topology section just after ligand parameters (Step 1) include at line 32
!sed -i "31a ; Include ligand topology\n#ifdef POSRES\n#include \"posre_\{drug}.itp\"\n#endif\n" topol.top

# # Step 3: Add the ligand molecule at line 62
# # First, find where the last molecule line is (i.e., "DNA 1") and add the drug molecule after that.
!sed -i "61a \{drug}                 1" topol.top

### Add box and add solvent (water)
`-d` can be change by [1.2_waterbox_setting.ipynb](1.2_waterbox_setting.ipynb)

In [7]:
!cp topol.top topol.top.bak1 

In [8]:
!cp topol.top.bak1 topol.top

In [9]:
!gmx editconf -f {jobname}.gro.pdb -o {jobname}_dode.pdb -c yes -d 0.727 -bt dodecahedron

                     :-) GROMACS - gmx editconf, 2023.2 (-:

Executable:   /home/user/opt/gromacs-2023/bin/gmx
Data prefix:  /home/user/opt/gromacs-2023
Working dir:  /home/user/workfolder/g4dna_protocol/propeller_poh/c.gromacs.run_md/1.create_system
Command line:
  gmx editconf -f 1kf1_POH.gro.pdb -o 1kf1_POH_dode.pdb -c yes -d 0.727 -bt dodecahedron

Note that major changes are planned in future for editconf, to improve usability and utility.
Read 807 atoms
No velocities found
    system size :  3.304  2.868  4.151 (nm)
    diameter    :  4.198               (nm)
    center      :  2.453 -0.131 -0.895 (nm)
    box vectors :  0.000  0.000  0.000 (nm)
    box angles  :   0.00   0.00   0.00 (degrees)
    box volume  :   0.00               (nm^3)
    shift       :  1.786  4.370  2.894 (nm)
new center      :  4.239  4.239  1.998 (nm)
new box vectors :  5.652  5.652  5.652 (nm)
new box angles  :  60.00  60.00  90.00 (degrees)
new box volume  : 127.69               (nm^3)

GROMACS reminds y

### Add solvent (water)

In [10]:
!gmx solvate -cp {jobname}_dode.pdb -cs spc216.gro -o {jobname}_water.pdb -p topol.top

                     :-) GROMACS - gmx solvate, 2023.2 (-:

Executable:   /home/user/opt/gromacs-2023/bin/gmx
Data prefix:  /home/user/opt/gromacs-2023
Working dir:  /home/user/workfolder/g4dna_protocol/propeller_poh/c.gromacs.run_md/1.create_system
Command line:
  gmx solvate -cp 1kf1_POH_dode.pdb -cs spc216.gro -o 1kf1_POH_water.pdb -p topol.top

Reading solute configuration
Reading solvent configuration

Initialising inter-atomic distances...

         based on residue and atom names, since they could not be
         definitively assigned from the information in your input
         files. These guessed numbers might deviate from the mass
         and radius of the atom type. Please check the output
         files if necessary. Note, that this functionality may
         be removed in a future GROMACS version. Please, consider
         using another file format for your input.

NOTE: From version 5.0 gmx solvate uses the Van der Waals radii
from the source below. This means the result

#### Add ions

## Remove PBC and See dodecahedron
select `0(System)` for output group

In [11]:
!echo "0" | gmx trjconv -s {jobname}_water.pdb -f {jobname}_water.pdb -o {jobname}_water.dode.pdb -pbc res -ur compact

                     :-) GROMACS - gmx trjconv, 2023.2 (-:

Executable:   /home/user/opt/gromacs-2023/bin/gmx
Data prefix:  /home/user/opt/gromacs-2023
Working dir:  /home/user/workfolder/g4dna_protocol/propeller_poh/c.gromacs.run_md/1.create_system
Command line:
  gmx trjconv -s 1kf1_POH_water.pdb -f 1kf1_POH_water.pdb -o 1kf1_POH_water.dode.pdb -pbc res -ur compact

Note that major changes are planned in future for trjconv, to improve usability and utility.
Will write pdb: Protein data bank file

         based on residue and atom names, since they could not be
         definitively assigned from the information in your input
         files. These guessed numbers might deviate from the mass
         and radius of the atom type. Please check the output
         files if necessary. Note, that this functionality may
         be removed in a future GROMACS version. Please, consider
         using another file format for your input.


         that are broken across periodic boundaries, t

In [12]:
## visualize
# !pymol {jobname}_water.dode.pdb

### Add ions
gmx genion ~
`-conc` unit: (mol/liters) default=0.125  
sele `5(SOL)` for group of solvent molecules

In [13]:
!gmx grompp -f {mdp_folder}/ions.mdp -c {jobname}_water.pdb -p topol.top -o ions.tpr

                      :-) GROMACS - gmx grompp, 2023.2 (-:

Executable:   /home/user/opt/gromacs-2023/bin/gmx
Data prefix:  /home/user/opt/gromacs-2023
Working dir:  /home/user/workfolder/g4dna_protocol/propeller_poh/c.gromacs.run_md/1.create_system
Command line:
  gmx grompp -f /home/user/workfolder/g4dna_protocol/smsl/opt/mdp/ions.mdp -c 1kf1_POH_water.pdb -p topol.top -o ions.tpr

Ignoring obsolete mdp entry 'ns_type'

NOTE 1 [file /home/user/workfolder/g4dna_protocol/smsl/opt/mdp/ions.mdp]:
  With Verlet lists the optimal nstlist is >= 10, with GPUs >= 20. Note
  that with the Verlet scheme, nstlist has no effect on the accuracy of
  your simulation.

Setting the LD random seed to 1045944110

Generated 5050 of the 5050 non-bonded parameter combinations
Generating 1-4 interactions: fudge = 0.5

Generated 5050 of the 5050 1-4 parameter combinations

Excluding 3 bonded neighbours molecule type 'DNA'

Excluding 3 bonded neighbours molecule type 'Ion2'

Excluding 3 bonded neighbours mol

In [14]:
!echo "SOL" | gmx genion -s ions.tpr -o {jobname}_solv.pdb -p topol.top -pname K -nname CL -neutral -conc 0.125

                      :-) GROMACS - gmx genion, 2023.2 (-:

Executable:   /home/user/opt/gromacs-2023/bin/gmx
Data prefix:  /home/user/opt/gromacs-2023
Working dir:  /home/user/workfolder/g4dna_protocol/propeller_poh/c.gromacs.run_md/1.create_system
Command line:
  gmx genion -s ions.tpr -o 1kf1_POH_solv.pdb -p topol.top -pname K -nname CL -neutral -conc 0.125

Reading file ions.tpr, VERSION 2023.2 (single precision)
Reading file ions.tpr, VERSION 2023.2 (single precision)
Will try to add 25 K ions and 10 CL ions.
Select a continuous group of solvent molecules
Group     0 (         System) has 12270 elements
Group     1 (            DNA) has   715 elements
Group     2 (              K) has     2 elements
Group     3 (            POH) has    90 elements
Group     4 (            Ion) has     2 elements
Group     5 (          Water) has 11463 elements
Group     6 (            SOL) has 11463 elements
Group     7 (      non-Water) has   807 elements
Group     8 ( Water_and_ions) has 11465 e

In [15]:
## rmpbc and visualize
## check System has 12200±1 elements
!echo "0" | gmx trjconv -s {jobname}_solv.pdb -f {jobname}_solv.pdb -o {jobname}_solv.dode.pdb -pbc res -ur compact
#!pymol {jobname}_ions.dode.pdb

                     :-) GROMACS - gmx trjconv, 2023.2 (-:

Executable:   /home/user/opt/gromacs-2023/bin/gmx
Data prefix:  /home/user/opt/gromacs-2023
Working dir:  /home/user/workfolder/g4dna_protocol/propeller_poh/c.gromacs.run_md/1.create_system
Command line:
  gmx trjconv -s 1kf1_POH_solv.pdb -f 1kf1_POH_solv.pdb -o 1kf1_POH_solv.dode.pdb -pbc res -ur compact

Note that major changes are planned in future for trjconv, to improve usability and utility.
Will write pdb: Protein data bank file

         based on residue and atom names, since they could not be
         definitively assigned from the information in your input
         files. These guessed numbers might deviate from the mass
         and radius of the atom type. Please check the output
         files if necessary. Note, that this functionality may
         be removed in a future GROMACS version. Please, consider
         using another file format for your input.


         that are broken across periodic boundaries, they

### Energy Minimization

In [16]:
!gmx grompp -f {mdp_folder}/min1.mdp -c {jobname}_solv.pdb -r {jobname}_solv.pdb -p topol.top -o min1.tpr
!gmx mdrun -deffnm min1 -c min1.pdb

                      :-) GROMACS - gmx grompp, 2023.2 (-:

Executable:   /home/user/opt/gromacs-2023/bin/gmx
Data prefix:  /home/user/opt/gromacs-2023
Working dir:  /home/user/workfolder/g4dna_protocol/propeller_poh/c.gromacs.run_md/1.create_system
Command line:
  gmx grompp -f /home/user/workfolder/g4dna_protocol/smsl/opt/mdp/min1.mdp -c 1kf1_POH_solv.pdb -r 1kf1_POH_solv.pdb -p topol.top -o min1.tpr

Setting the LD random seed to 2111798782

Generated 5050 of the 5050 non-bonded parameter combinations
Generating 1-4 interactions: fudge = 0.5

Generated 5050 of the 5050 1-4 parameter combinations

Excluding 3 bonded neighbours molecule type 'DNA'

Excluding 3 bonded neighbours molecule type 'Ion2'

Excluding 3 bonded neighbours molecule type 'POH'

Excluding 2 bonded neighbours molecule type 'SOL'

Excluding 1 bonded neighbours molecule type 'K'

Excluding 1 bonded neighbours molecule type 'CL'

NOTE 1 [file topol.top, line 65]:
  In moleculetype 'Ion2' 2 atoms are not bound by a pot

In [17]:
## -r argument min1.pdb or {jobname}_solv.pdb
!gmx grompp -f {mdp_folder}/min2.mdp -c min1.pdb -r {jobname}_solv.pdb -p topol.top -o min2.tpr
!gmx mdrun -deffnm min2 -c min2.pdb

                      :-) GROMACS - gmx grompp, 2023.2 (-:

Executable:   /home/user/opt/gromacs-2023/bin/gmx
Data prefix:  /home/user/opt/gromacs-2023
Working dir:  /home/user/workfolder/g4dna_protocol/propeller_poh/c.gromacs.run_md/1.create_system
Command line:
  gmx grompp -f /home/user/workfolder/g4dna_protocol/smsl/opt/mdp/min2.mdp -c min1.pdb -r 1kf1_POH_solv.pdb -p topol.top -o min2.tpr


NOTE 1 [file /home/user/workfolder/g4dna_protocol/smsl/opt/mdp/min2.mdp]:
  With Verlet lists the optimal nstlist is >= 10, with GPUs >= 20. Note
  that with the Verlet scheme, nstlist has no effect on the accuracy of
  your simulation.

Setting the LD random seed to 2132729567

Generated 5050 of the 5050 non-bonded parameter combinations
Generating 1-4 interactions: fudge = 0.5

Generated 5050 of the 5050 1-4 parameter combinations

Excluding 3 bonded neighbours molecule type 'DNA'

turning H bonds into constraints...

Excluding 3 bonded neighbours molecule type 'Ion2'

turning H bonds into co

## Make Index File

In [18]:
pdbAtom = !grep "ATOM" {pdbid}.gro.pdb
n_atom = len(pdbAtom)

In [19]:
cmd = f'''\
a 1-{n_atom}
name 10 Model
q
'''
!echo -e "{cmd}" | gmx make_ndx -f min2.pdb -o {jobname}.simulate.ndx

                     :-) GROMACS - gmx make_ndx, 2023.2 (-:

Executable:   /home/user/opt/gromacs-2023/bin/gmx
Data prefix:  /home/user/opt/gromacs-2023
Working dir:  /home/user/workfolder/g4dna_protocol/propeller_poh/c.gromacs.run_md/1.create_system
Command line:
  gmx make_ndx -f min2.pdb -o 1kf1_POH.simulate.ndx


Reading structure file
Going to read 0 old index file(s)
Analysing residue names:
There are:    22        DNA residues
There are:    37        Ion residues
There are:     1      Other residues
There are:  3786      Water residues
Analysing residues not classified as Protein/DNA/RNA/Water and splitting into groups...

  0 System              : 12200 atoms
  1 DNA                 :   715 atoms
  2 K                   :    27 atoms
  3 POH                 :    90 atoms
  4 CL                  :    10 atoms
  5 Ion                 :    37 atoms
  6 Water               : 11358 atoms
  7 SOL                 : 11358 atoms
  8 non-Water           :   842 atoms
  9 Water_and_ions  

### Equilibrium (generate [1.3_equilibrium.slurm](1.3_equilibrium.slurm))
(In the energy minimization process, the only parameter required is the initial structure. However, for NVT or NPT processes, since the .pdb file does not contain information about speed or temperature state, an additional parameter, "-t", must be included to provide this information.)

In [20]:
slurm_script = f'''\
#!/bin/bash
#SBATCH --partition=gpu                  # Define the partition on which the job shall run
#SBATCH --job-name={jobname}                  # Job name
#SBATCH --ntasks=1                       # Total number of tasks
#SBATCH --nodes=1                        # Number of nodes on which to run
#SBATCH --cpus-per-task=20               # Threads per task
#SBATCH --gres=gpu:2                     # number of GPUs per node (gres=gpu:N)
#SBATCH --mem=50000                      # Memory per job in MB
#SBATCH --time=0-999:00:00               # Time limit ( dd-hh:mm:ss ) ← e.g. Maximum run time: 1 hour
#SBATCH --time=0-999:00:00               # Time limit ( dd-hh:mm:ss ) ← e.g. Maximum run time: 1 hour
#SBATCH --output=slurm_simulation_%j.out       # Standard output log
#SBATCH --error=slurm_simulation_%j.out        # Standard error log

  echo "Start at `date`"
  echo "Running on hosts: $SLURM_NODELIST"
  echo "Running on $SLURM_NNODES nodes."
  echo "Running $SLURM_NTASKS tasks."
  echo "Using ${{SLURM_CPUS_PER_TASK}} cpus per task"
  echo "Job id is $SLURM_JOBID"
  echo "Current working directory is `pwd`"

mdpFolder="{mdp_folder}"
mdFolder="."

## load program
export PATH=$HOME/opt/gcc-10.4.0/bin:$PATH
export LD_LIBRARY_PATH=$HOME/opt/gcc-10.4.0/lib64
source $HOME/opt/gromacs-2023/bin/GMXRC

cd ${{mdFolder}}
## HEAT
gmx grompp -f ${{mdpFolder}}/heat.mdp -c min2.pdb -r {jobname}_solv.pdb -p topol.top -o heat.tpr
gmx mdrun -deffnm heat -c heat.pdb -ntmpi 4 -nb gpu -pme gpu -npme 1 -gputasks 0011 -ntomp 6
## NVT
gmx grompp -f ${{mdpFolder}}/nvt1.mdp -c heat.pdb -r {jobname}_solv.pdb -t heat.cpt -p topol.top -o nvt1.tpr
gmx mdrun -deffnm nvt1 -c nvt1.pdb -ntmpi 4 -nb gpu -pme gpu -npme 1 -gputasks 0011 -ntomp 6
## NPT1
gmx grompp -f ${{mdpFolder}}/npt1.mdp -c nvt1.pdb -r {jobname}_solv.pdb -t nvt1.cpt -p topol.top -o npt1.tpr
gmx mdrun -deffnm npt1 -c npt1.pdb -ntmpi 4 -nb gpu -pme gpu -npme 1 -gputasks 0011 -ntomp 6
## NPT2
gmx grompp -f ${{mdpFolder}}/npt2.mdp -c npt1.pdb -r {jobname}_solv.pdb -t npt1.cpt -p topol.top -o npt2.tpr
gmx mdrun -deffnm npt2 -c npt2.pdb -ntmpi 4 -nb gpu -pme gpu -npme 1 -gputasks 0011 -ntomp 6
'''

open("1.3_equilibrium.slurm", 'w').write(slurm_script)

2177

#### Running slurm command:
```
> sbatch 1.3_equilibrium.slurm
```